# Introduction

<p>We want to download data to support a data visualization analysis  of the best movies of every decade between 1920 and 2010. We will use the IMDb python package to obtain information about the movies. This package doesn’t have a function that returns the top rated films of every decade so we have to construct an initial set of films, download them all and then select the top rated ones from every decade. </p>



# Initial dataset

<p>We consider an initial subset of 40’000 movies from Film Tv. We drop the movies with a number of votes below the median and then we select the top 100  movies by expert’s rating from every decade, this will be our initial set. The experts on Film Tv are the most active users and those with the most competent reviews in the opinion of the site managers.</p>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
movies = pd.read_csv('filmtv.csv', header=[0])

In [ ]:
#select only columns of interest and drop null values
movies =  movies[['filmtv_id', 'title', 'year', 'genre', 'duration', 'country', 'avg_vote', 'critics_vote', 'public_vote', 'total_votes', 'description']]
movies = movies.dropna()

In [ ]:
import statistics
movies = movies[movies['total_votes'] >= statistics.median(movies['total_votes'])]

In [ ]:
movies['decade']=(movies['year']//10*10)

#consider only the movies in the decades between 1920 and 2010
movies = movies[(movies['decade'] >= 1920) & (movies['decade'] < 2020)]
decades = movies['decade'].unique()

top_list = []

for decade in decades:
  bests = movies[movies['decade']==decade].sort_values(by=['critics_vote'], ascending=False).head(100)
  best_list = bests['title'].tolist()

  top_list = top_list + best_list

boolean_series = movies.title.isin(top_list)

#top movies will containt only the top 100 films of every decade
top_movies = movies[boolean_series]

# IMDb

<p> The function of the IMDb python package that gives informations about the movies recieves as input the IMDb ID of the film considered so we obtain all the IMDB IDs of the movies of the initial set, through another function of the package, and we save them in a list called top_IDS. </p>

In [ ]:
!pip install IMDbPY

In [ ]:
import imdb

ia = imdb.IMDb()
top_IDS=[]

for film in top_list:
  try :
    search = ia.search_movie(film)

    if isinstance(search, list):
      top_IDS.append(search[0].movieID)
  
  except:
    continue


<p> We will build three different datasets to support three different analysis. The first will focus on the top 5 rated films of each decade and their budgets and grosses. The second will focus on the genres of the films. The third will focus on the countries where the films are produced. </p>

# Top films
<p> For every movie in top IDS we download the title, the year, the number of votes, the plot, the budget, the gross and the rating. We consider only the films that have all these informations.</p>

In [ ]:
titles=[]
gross=[]
rating=[]
votes=[]
year=[]
plots=[]

for id in top_IDS:
  film = ia.get_movie(id)
  if ( ('plot' in film.keys()) & ('kind' in film.keys()) & ('title' in film.keys()) & ('box office' in film.keys()) & ('rating' in film.keys()) & ('votes' in film.keys()) & ('year' in film.keys()) ):
    if(film['kind'] == 'movie'):
      titles.append(film['title'])
      rating.append(film['rating'])
      gross.append(film['box office'])
      votes.append(film['votes'])
      year.append(film['year'])
      plots.append(film['plot'])

#'box office' contiene varie informazioni al suo interno seleziono quelle di interesse
budgets=[]
for i in range(len(gross)):
  budgets.append(gross[i].get('Budget'))

grosses=[]
for i in range(len(gross)):
  grosses.append(gross[i].get('Cumulative Worldwide Gross'))

#creo il datframe che mi servirà per il grafico
df = pd.DataFrame(list(zip(titles, year, rating, votes, grosses, budgets, plots)),columns =['title', 'year','rating','votes','grosses','budgets', 'plot'])

<p> We select the top 5 rated films for each decade </p>.

In [ ]:
#aggiungo la decade perché li studierò per decade
df['decade']=(df['year']//10*10)

#prendo i top5 per rating di ogni decade, dal 1920 ad oggi
decades = df['decade'].unique()

top_list = []

for decade in decades:
  bests = df[df['decade']==decade].sort_values(by=['rating'], ascending=False).head(5)
  best_list = bests['title'].tolist()

  top_list = top_list + best_list

boolean_series = df.title.isin(top_list)
df_top = df[boolean_series]

df_top.reset_index(drop=True, inplace=True)
df_top

<p> We observe that many films have None value for the grosses field. By checking on the website we found out that for some films earnings are not reported, so we decide to get the economic information about the films from Wikipedia instead of IMDb. We will consider the budgets of the films and the worldwide box offices, instead of the grosses given by IMDb. </p>

# Genres 
<p> For every movie in top IDS we download the title, the year, the genre and the rating. We consider only the films that have all these informations.</p>

In [ ]:
titles_genre =[]
years_genre = []
genre = []
rating_genre = []

for id in top_IDS:
  film = ia.get_movie(id)
  if ( ('title' in film.keys()) & ('genres' in film.keys()) & ('year' in film.keys()) & ('rating' in film.keys()) &('kind' in film.keys())):
    if(film['kind'] == 'movie'):
      titles_genre.append(film['title'])
      years_genre.append(film['year'])
      genre.append(film['genre'])
      rating_genre.append(film['rating'])

In [ ]:
genres = pd.DataFrame(list(zip(titles_genre, years_genre, genre, rating_genre)),columns =['title', 'year', 'genre', 'rating'])
genres['genre'] = genres['genre'].astype(str)

for i in range(len(genres)):
  genres.loc[i,'genre']=genres.loc[i,'genre'].replace('\'','')
  genres.loc[i,'genre']=genres.loc[i,'genre'].replace('[','')
  genres.loc[i,'genre']=genres.loc[i,'genre'].replace(']','')

In [ ]:
genres['decade']=(genres['year']//10*10)

In [ ]:
px.histogram(genres, x='decade')

<p>For the decade of 1920 we have only 47 films with complete information. We therefore choose to consider the best 45 films of each decade according to IMDb rating.</p>

In [ ]:
genres = genres[genres['decade'] < 2020]
decades = genres['decade'].unique()

top_list = []

for decade in decades:
  bests = genres[genres['decade']==decade].sort_values(by=['rating'], ascending=False).head(45)
  best_list = bests['title'].tolist()

  top_list = top_list + best_list

boolean_series = genres.title.isin(top_list)
top_genres = genres[boolean_series]

We observe that each film can have several genres. We decide to keep only the first genre indicated for each film since we consider it as the main genre.

In [ ]:
#tengo solo il primo genere per ogni film
for i in range(len(top_genres)):
  if (len(top_genres.loc[i, 'genre'].split(',')) > 0):
    top_genres.loc[i, 'genre'] = top_genres.loc[i, 'genre'].split(',')[0] 

Now we build a dataset that indicates the number of films in each decade for every genre considered.

In [ ]:
all_rec_df = [pd.Series(row['year'],row['genre'].split(", "))              
                    for _, row in top_genres.iterrows()]
all_rec_df = pd.concat(all_rec_df).reset_index()

all_rec_df.columns = ['genre','year']
all_rec_df['year'] = all_rec_df['year'].astype('str')
all_genres = all_rec_df

#in questa tabella i generi sono le colonne e gli anni gli indici di riga
year_genre = pd.pivot_table(all_genres,  index=['year'], columns=['genre'], aggfunc= len,fill_value=0)

# #ottengo il totle per ogni anno
year_genre['Total'] = year_genre.sum(axis = 1)
year_genre

# #metto year anche come colonna non solo come indice
year_genre['year'] = year_genre.index
year_genre['year'] = pd.to_numeric(year_genre['year'],errors = 'coerce')

#ragruppo per decadi
decade_year_genre = year_genre.groupby((year_genre['year']//10)*10).sum()
decade_year_genre['year'] = decade_year_genre.index

#questa è la tabella che voglio io
genres_by_year = year_genre.melt(id_vars=['year', 'Total'], var_name='Genre', value_name='Value')

decade_year_genre = year_genre.groupby((year_genre['year']//10)*10).sum()
decade_year_genre['year'] = decade_year_genre.index
genres_by_decade = decade_year_genre.melt(id_vars=['year', 'Total'], var_name='Genre', value_name='Value')

# #aggiungo le percentuali sul totale della decade per ogni genere
genres_by_decade['percent'] = genres_by_decade['Value']/genres_by_decade['Total']*100

In [ ]:
genres_by_decade

# Countries
<p> For every movie in top IDS we download the title, the year, the countries of production and the rating. We consider only the films that have all these informations.</p>

In [ ]:
countries = []
years_countries = []
titles_countries = []
rating_countries = []

for id in top_IDS:
  film = ia.get_movie(id)

  if ( ('title' in film.keys()) & ('countries' in film.keys()) & ('year' in film.keys()) & ('rating' in film.keys()) & ('kind' in film.keys()) ):
    if(film['kind'] == 'movie'):
      countries.append(film['countries'])
      years_countries.append(film['year'])
      titles_countries.append(film['title'])
      rating_countries.append(film['rating'])

In [ ]:
countries = pd.DataFrame(list(zip(titles_countries, years_countries, countries, rating_countries)),columns =['title', 'year', 'country', 'rating'])
countries['country'] = countries['country'].astype(str)

for i in range(len(countries)):
  countries.loc[i,'country']=countries.loc[i,'country'].replace('\'','')
  countries.loc[i,'country']=countries.loc[i,'country'].replace('[','')
  countries.loc[i,'country']=countries.loc[i,'country'].replace(']','')

In [ ]:
countries['decade']=(countries['year']//10*10)

In [ ]:
px.histogram(countries, x="decade")

<p>For the decade of 1920 we have only 47 films with complete information. We therefore choose to consider the best 45 films of each decade according to IMDb rating.</p>

In [ ]:
countries = countries[countries['decade'] < 2020]
decades = countries['decade'].unique()

top_list = []

for decade in decades:
  bests = countries[countries['decade']==decade].sort_values(by=['rating'], ascending=False).head(45)
  best_list = bests['title'].tolist()

  top_list = top_list + best_list

boolean_series = countries.title.isin(top_list)
top_countries = countries[boolean_series]

Now we build a dataset that indicates the number of films in each decade in which every country has participated.

In [ ]:
#serie i cui valori sono gli anni e i generi considerati separatamente, quindi un film può appartenere a più generi
all_rec_df = [pd.Series(row['year'],row['country'].split(", "))              
                    for _, row in top_countries.iterrows()]
all_rec_df = pd.concat(all_rec_df).reset_index()

#creo il datframe con le colonne genere e anno
all_rec_df.columns = ['country','year']
all_rec_df['year'] = all_rec_df['year'].astype('str')
all_countrys = all_rec_df

#in questa tabella i generi sono le colonne e gli anni gli indici di riga
year_country = pd.pivot_table(all_countrys,  index=['year'], columns=['country'], aggfunc= len,fill_value=0)
year_country
# #ottengo il totle per ogni anno
year_country['Total'] = year_country.sum(axis = 1)
year_country

# #metto year anche come colonna non solo come indice
year_country['year'] = year_country.index
year_country['year'] = pd.to_numeric(year_country['year'],errors = 'coerce')

#ragruppo per decadi
decade_year_country = year_country.groupby((year_country['year']//10)*10).sum()
decade_year_country['year'] = decade_year_country.index
decade_year_country

#questa è la tabella che voglio io
countrys_by_year = year_country.melt(id_vars=['year', 'Total'], var_name='country', value_name='Value')

decade_year_country = year_country.groupby((year_country['year']//10)*10).sum()
decade_year_country['year'] = decade_year_country.index
countries_by_decade = decade_year_country.melt(id_vars=['year', 'Total'], var_name='country', value_name='Value')

# #aggiungo le percentuali sul totale della decade per ogni genere
#countries_by_decade['percent'] = countries_by_decade['Value']/countries_by_decade['Total']*100

#IL TOTALE NON è IL TOTALE DI FILM PERCHè OGNI FILM CONTA 1 PER TUTTE LE NAZIONI CHE HANNO PARTECIPATO AL FILM

In [ ]:
countries_by_decade = countries_by_decade.drop('Total', axis=1)
countries_by_decade['total'] = 45
countries_by_decade['total'][countries_by_decade['year'] == 1970] = 46
countries_by_decade['total'][countries_by_decade['year'] == 2000] = 46
countries_by_decade['percent'] = countries_by_decade['Value'] / countries_by_decade['total']